Módulo DEAP --- descobrindo a senha
===================================



Vamos resolver o problema da senha com o `DEAP`. O início da resolução é similar.



In [1]:
import random
from string import ascii_lowercase, ascii_uppercase, digits
import numpy as np
from deap import base
from deap import tools
from deap import creator
from deap.algorithms import eaSimple

from funcoes_7 import gene_senha

In [2]:
SENHA = list("TorresmoPururuca2005")
CARACTERES_POSSIVEIS = ascii_lowercase + ascii_uppercase + digits
TAMANHO_SENHA = len(SENHA)

TAMANHO_POPULACAO = 100
NUM_GERACOES = 500
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.15
CHANCE_DE_MUTACAO_POR_GENE = 0.25
TAMANHO_TORNEIO = 3
TAMANHO_HALL_DA_FAMA = 1

In [3]:
def funcao_objetivo_senha(candidato, senha_verdadeira):
    """Computa a funcao objetivo de um candidato no problema da senha

    Args:
      candidato: lista contendo as letras da senha
      senha_verdadeira: a senha que você está tentando descobrir

    """
    distancia = 0

    for letra_candidato, letra_senha in zip(candidato, senha_verdadeira):
        num_letra_candidato = ord(letra_candidato)
        num_letra_senha = ord(letra_senha)
        distancia += abs(num_letra_candidato - num_letra_senha)

    return (distancia, )

O `DEAP` não tem uma função de mutação de caracteres, precisamos criar uma! Tem um pequeno porém, o `DEAP` espera que a função de mutação retorne um indivíduo dentro de uma tupla. Essa é a realidade, temos que aceitar e seguir.



In [4]:
def mutacao_senha(individuo, caracteres_possiveis):
    """Realiza a mutação de um gene no problema da senha.

    Args:
      individuo: uma lista representado um individuo no problema da senha
      caracteres_possiveis: caracteres possíveis de serem sorteados.

    """
    gene = random.randint(0, len(individuo) - 1)
    individuo[gene] = gene_senha(caracteres_possiveis)
    return (individuo, )

Como nosso problema agora é de <u>minimização</u>, precisamos alterar a forma que computamos o *fitness*. Observe o `weights=(-1.0,)` que é diferente do que estávamos fazendo antes nos problemas de maximização.



In [5]:
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))

O restante é bem similar ao que já fizemos anteriormente.



In [6]:
creator.create("Individuo", list, fitness=creator.FitnessMin)

toolbox = base.Toolbox()

toolbox.register("cria_gene", gene_senha, CARACTERES_POSSIVEIS)

toolbox.register(
    "cria_individuo",
    tools.initRepeat,
    creator.Individuo,
    toolbox.cria_gene,
    TAMANHO_SENHA,
)

toolbox.register(
    "populacao", tools.initRepeat, list, toolbox.cria_individuo, TAMANHO_POPULACAO
)

toolbox.register(
    "select", tools.selTournament, tournsize=TAMANHO_TORNEIO
)

O cálculo da função objetivo é realizado pela `funcao_objetivo_senha`. Observe que esta função requer um argumento, diferente do que observamos nos problemas anteriores. Precisamos levar isso em consideração quando formos criar a ferramenta `evaluate`.



In [7]:
toolbox.register("evaluate", funcao_objetivo_senha, senha_verdadeira=SENHA)

Vamos usar o cruzamento uniforme neste problema. Este cruzamento requer o argumento `indpb` que representa a chance de cada gene ser trocado entre o pai e a mãe. Vamos considerar uma chance de 50%.



In [8]:
toolbox.register("mate", tools.cxUniform, indpb=0.5)

Aqui usamos a função de mutação que criamos lá em cima.



In [9]:
toolbox.register(
    "mutate",
    mutacao_senha,
    caracteres_possiveis=CARACTERES_POSSIVEIS
)

O restante do código permanece como estava.



In [10]:
hall_da_fama = tools.HallOfFame(TAMANHO_HALL_DA_FAMA)

estatisticas = tools.Statistics(lambda ind: ind.fitness.values)
estatisticas.register("média", np.mean)
estatisticas.register("desv. padrão", np.std)
estatisticas.register("min", np.min)
estatisticas.register("max", np.max)

populacao_inicial = toolbox.populacao()

populacao_final, log = eaSimple(
    populacao_inicial,
    toolbox,
    cxpb=CHANCE_DE_CRUZAMENTO,
    mutpb=CHANCE_DE_MUTACAO,
    ngen=NUM_GERACOES,
    stats=estatisticas,
    halloffame=hall_da_fama,
    verbose=True,
)

gen	nevals	média	desv. padrão	min	max
0  	100   	547.5	99.0228     	260	778
1  	64    	466.07	78.0052     	237	621
2  	51    	410.07	69.9696     	221	641
3  	55    	356.62	56.4825     	221	483
4  	63    	313.1 	46.1436     	184	440
5  	60    	276.46	43.2577     	184	415
6  	67    	240.99	31.0189     	176	319
7  	59    	214.83	26.8809     	159	283
8  	59    	192.89	25.0152     	136	256
9  	51    	175.38	25.8452     	123	241
10 	62    	156.36	19.9397     	110	200
11 	57    	141.63	18.7316     	108	214
12 	56    	129.93	16.5434     	98 	191
13 	65    	119.53	15.4993     	90 	179
14 	69    	112.4 	14.4284     	83 	150
15 	50    	102.35	12.26       	76 	145
16 	58    	94.27 	11.5038     	70 	145
17 	59    	89.58 	12.8773     	69 	146
18 	70    	84.42 	13.45       	64 	133
19 	54    	78.38 	12.6252     	64 	147
20 	61    	74.14 	10.9517     	64 	131
21 	71    	71.84 	9.9335      	63 	131
22 	59    	71.65 	13.3089     	61 	131
23 	61    	71.11 	13.6959     	58 	129
24 	69    	65.97 	8.33961  

In [11]:
# print(log)

In [12]:
print("".join(hall_da_fama.items[0]))

TorresmoPururuca2005
